In [1]:
# We use a repo publicly available to train haar-cascade classifier
!git clone https://github.com/mrnugget/opencv-haar-classifier-training.git

fatal: destination path 'opencv-haar-classifier-training' already exists and is not an empty directory.


In [2]:
# We bring over our data
!git clone https://github.com/varunjain3/EigenFaces.git

Cloning into 'EigenFaces'...
Updating files:  12% (542/4468)
Updating files:  13% (581/4468)
Updating files:  14% (626/4468)
Updating files:  15% (671/4468)
Updating files:  16% (715/4468)
Updating files:  17% (760/4468)
Updating files:  18% (805/4468)
Updating files:  19% (849/4468)
Updating files:  20% (894/4468)
Updating files:  21% (939/4468)
Updating files:  22% (983/4468)
Updating files:  23% (1028/4468)
Updating files:  24% (1073/4468)
Updating files:  25% (1117/4468)
Updating files:  25% (1143/4468)
Updating files:  26% (1162/4468)
Updating files:  27% (1207/4468)
Updating files:  27% (1228/4468)
Updating files:  28% (1252/4468)
Updating files:  29% (1296/4468)
Updating files:  30% (1341/4468)
Updating files:  31% (1386/4468)
Updating files:  32% (1430/4468)
Updating files:  33% (1475/4468)
Updating files:  34% (1520/4468)
Updating files:  35% (1564/4468)
Updating files:  36% (1609/4468)
Updating files:  37% (1654/4468)
Updating files:  38% (1698/4468)
Updating files:  39% (174

In [3]:
# Copying our positive images to the training repo
!cp -r /content/EigenFaces/data/pos/* /content/opencv-haar-classifier-training/positive_images/

# Copying negative images
!cp -r /content/EigenFaces/data/neg/* /content/opencv-haar-classifier-training/negative_images/

'cp' is not recognized as an internal or external command,
operable program or batch file.
'cp' is not recognized as an internal or external command,
operable program or batch file.


In [4]:
# Building Opencv
!wget http://downloads.sourceforge.net/project/opencvlibrary/opencv-unix/2.4.9/opencv-2.4.9.zip
!unzip opencv-2.4.9.zip >/dev/null

'wget' is not recognized as an internal or external command,
operable program or batch file.
The system cannot find the path specified.


In [5]:
%cd "opencv-haar-classifier-training"

C:\Users\SilC\A hybrid EigenFace and Viola Jones algorithm\EigenFaces-master\opencv-haar-classifier-training


In [6]:
# Making positive labels
!find ./positive_images -iname "*.jpg" > positives.txt

Access denied - ./POSITIVE_IMAGES
File not found - -INAME


In [7]:
# Making negative labels
!find ./negative_images -iname "*.jpg" > negatives.txt

Access denied - ./NEGATIVE_IMAGES
File not found - -INAME


In [8]:
# Creating .vec file
 !perl bin/createsamples.pl positives.txt negatives.txt samples 1000\
   "opencv_createsamples -bgcolor 0 -bgthresh 0 -maxxangle 1.1\
   -maxyangle 1.1 maxzangle 0.5 -maxidev 40 -w 40 -h 40" >/dev/null

IndentationError: unexpected indent (3277867665.py, line 2)

In [14]:
# Change due to python3, orignal file was for python2

%%writefile /content/opencv-haar-classifier-training/tools/mergevec.py

###############################################################################
# Copyright (c) 2014, Blake Wulfe
#
# Permission is hereby granted, free of charge, to any person obtaining a copy
# of this software and associated documentation files (the "Software"), to deal
# in the Software without restriction, including without limitation the rights
# to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
# copies of the Software, and to permit persons to whom the Software is
# furnished to do so, subject to the following conditions:
#
# The above copyright notice and this permission notice shall be included in
# all copies or substantial portions of the Software.
#
# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
# AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
# OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN
# THE SOFTWARE.
###############################################################################

"""
File: mergevec.py
Author: blake.w.wulfe@gmail.com
Date: 6/13/2014
File Description:

	This file contains a function that merges .vec files called "merge_vec_files".
	I made it as a replacement for mergevec.cpp (created by Naotoshi Seo.
	See: http://note.sonots.com/SciSoftware/haartraining/mergevec.cpp.html)
	in order to avoid recompiling openCV with mergevec.cpp.

	To use the function:
	(1) Place all .vec files to be merged in a single directory (vec_directory).
	(2) Navigate to this file in your CLI (terminal or cmd) and type "python mergevec.py -v your_vec_directory -o your_output_filename".

		The first argument (-v) is the name of the directory containing the .vec files
		The second argument (-o) is the name of the output file

	To test the output of the function:
	(1) Install openCV.
	(2) Navigate to the output file in your CLI (terminal or cmd).
	(2) Type "opencv_createsamples -w img_width -h img_height -vec output_filename".
		This should show the .vec files in sequence.

"""

import sys
import glob
import struct
import argparse
import traceback


def exception_response(e):
	exc_type, exc_value, exc_traceback = sys.exc_info()
	lines = traceback.format_exception(exc_type, exc_value, exc_traceback)
	for line in lines:
		print(line)

def get_args():
	parser = argparse.ArgumentParser()
	parser.add_argument('-v', dest='vec_directory')
	parser.add_argument('-o', dest='output_filename')
	args = parser.parse_args()
	return (args.vec_directory, args.output_filename)

def merge_vec_files(vec_directory, output_vec_file):
	"""
	Iterates throught the .vec files in a directory and combines them.

	(1) Iterates through files getting a count of the total images in the .vec files
	(2) checks that the image sizes in all files are the same

	The format of a .vec file is:

	4 bytes denoting number of total images (int)
	4 bytes denoting size of images (int)
	2 bytes denoting min value (short)
	2 bytes denoting max value (short)

	ex: 	6400 0000 4605 0000 0000 0000

		hex		6400 0000  	4605 0000 		0000 		0000
			   	# images  	size of h * w		min		max
		dec	    	100     	1350			0 		0

	:type vec_directory: string
	:param vec_directory: Name of the directory containing .vec files to be combined.
				Do not end with slash. Ex: '/Users/username/Documents/vec_files'

	:type output_vec_file: string
	:param output_vec_file: Name of aggregate .vec file for output.
		Ex: '/Users/username/Documents/aggregate_vec_file.vec'

	"""

	# Check that the .vec directory does not end in '/' and if it does, remove it.
	if vec_directory.endswith('/'):
		vec_directory = vec_directory[:-1]
	# Get .vec files
	files = glob.glob('{0}/*.vec'.format(vec_directory))

	# Check to make sure there are .vec files in the directory
	if len(files) <= 0:
		print('Vec files to be mereged could not be found from directory: {0}'.format(vec_directory))
		sys.exit(1)
	# Check to make sure there are more than one .vec files
	if len(files) == 1:
		print('Only 1 vec file was found in directory: {0}. Cannot merge a single file.'.format(vec_directory))
		sys.exit(1)


	# Get the value for the first image size
	prev_image_size = 0
	try:
		with open(files[0], 'rb') as vecfile:
			content = b''.join((line) for line in vecfile.readlines())
			val = struct.unpack('<iihh', content[:12])
			prev_image_size = val[1]
	except IOError as e:
		print('An IO error occured while processing the file: {0}'.format(f))
		exception_response(e)


	# Get the total number of images
	total_num_images = 0
	for f in files:
		try:
			with open(f, 'rb') as vecfile:
				content = b''.join((line) for line in vecfile.readlines())
				val = struct.unpack('<iihh', content[:12])
				num_images = val[0]
				image_size = val[1]
				if image_size != prev_image_size:
					err_msg = """The image sizes in the .vec files differ. These values must be the same. \n The image size of file {0}: {1}\n
						The image size of previous files: {0}""".format(f, image_size, prev_image_size)
					sys.exit(err_msg)

				total_num_images += num_images
		except IOError as e:
			print('An IO error occured while processing the file: {0}'.format(f))
			exception_response(e)


	# Iterate through the .vec files, writing their data (not the header) to the output file
	# '<iihh' means 'little endian, int, int, short, short'
	header = struct.pack('<iihh', total_num_images, image_size, 0, 0)
	try:
		with open(output_vec_file, 'wb') as outputfile:
			outputfile.write(header)

			for f in files:
				with open(f, 'rb') as vecfile:
					content = b''.join((line) for line in vecfile.readlines())
					outputfile.write(bytearray(content[12:]))
	except Exception as e:
		exception_response(e)


if __name__ == '__main__':
	vec_directory, output_filename = get_args()
	if not vec_directory:
		sys.exit('mergvec requires a directory of vec files. Call mergevec.py with -v /your_vec_directory')
	if not output_filename:
		sys.exit('mergevec requires an output filename. Call mergevec.py with -o your_output_filename')

	merge_vec_files(vec_directory, output_filename)

UsageError: Line magic function `%%writefile` not found.


In [15]:
# Calling the file updated above
!python ./tools/mergevec.py -v samples/ -o samples.vec

python: can't open file 'C:\Users\SilC\A hybrid EigenFace and Viola Jones algorithm\EigenFaces-master\tools\mergevec.py': [Errno 2] No such file or directory


In [16]:
# Training the classifier
!rm -r classifier
!mkdir classifier

!opencv_traincascade -data classifier -vec samples.vec -bg negatives.txt\
   -numStages 10 -minHitRate 0.999 -maxFalseAlarmRate 0.5 -numPos 300\
   -numNeg 550 -w 40 -h 40 -mode ALL -precalcValBufSize 1024\
   -precalcIdxBufSize 1024

'rm' is not recognized as an internal or external command,
operable program or batch file.
A subdirectory or file classifier already exists.
'opencv_traincascade' is not recognized as an internal or external command,
operable program or batch file.
